In [ ]:
from jinjasql import JinjaSql
j = JinjaSql(param_style='pyformat')

In [ ]:
template = """
    SELECT project, timesheet, hours
    FROM timesheet
    WHERE user_id = {{ user_id }}
    {% if project_id %}
    AND project_id = {{ project_id }}
    {% endif %}
"""

In [ ]:
template

'\n    SELECT project, timesheet, hours\n    FROM timesheet\n    WHERE user_id = {{ user_id }}\n    {% if project_id %}\n    AND project_id = {{ project_id }}\n    {% endif %}\n'

# Dynamic `WHERE` Clause 

In [ ]:
data = {
    "project_id": 123,
    "user_id": u"sripathi"
}

query, bind_params = j.prepare_query(template, data)
print(query)


    SELECT project, timesheet, hours
    FROM timesheet
    WHERE user_id = %(user_id_1)s
    
    AND project_id = %(project_id_2)s
    


In [ ]:
data = {
#     "project_id": 123,
    "user_id": u"sripathi"
}

query, bind_params = j.prepare_query(template, data)
print(query)


    SELECT project, timesheet, hours
    FROM timesheet
    WHERE user_id = %(user_id_1)s
    


Notice that the `AND` clause is automatically removed using the `Jinja` if/else condition.

In [ ]:
query

'\n    SELECT project, timesheet, hours\n    FROM timesheet\n    WHERE user_id = %(user_id_1)s\n    '

In [ ]:
query %bind_params

'\n    SELECT project, timesheet, hours\n    FROM timesheet\n    WHERE user_id = sripathi\n    '

In [ ]:
bind_params

{'user_id_1': 'sripathi'}

In [ ]:
from six import string_types

def quote_sql_string(value):
    '''
    If `value` is a string type, escapes single quotes in the string
    and returns the string enclosed in single quotes.
    '''
    if isinstance(value, string_types):
        new_value = str(value)
        new_value = new_value.replace("'", "''")
        return "'{}'".format(new_value)
    return value

from copy import deepcopy
def get_sql_from_template(query, bind_params=None):
    if not bind_params:
        return query
    params = deepcopy(bind_params)
    for key, val in params.items():
        params[key] = quote_sql_string(val)
    return query % params

In [ ]:
q = """
select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = %(user_id)s
    and transaction_date = %(transaction_date)s
group by
    user_id
    """

In [ ]:
print(get_sql_from_template(q, bind_params={'user_id':'1','transaction_date':'123'}))


select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = '1'
    and transaction_date = '123'
group by
    user_id
    


# Reference
* https://towardsdatascience.com/a-simple-approach-to-templated-sql-queries-in-python-adc4f0dc511